In [1]:
""" 3/27/18 
With a functional mister_ed package, it's time to run the madry challenge.

Step 1 is to build a defended network that can transfer to their tensorflow nets.
Because Madry uses tensorflow, transferability is a must. 

From Madry:
The model is a residual convolutional neural network consisting of five residual units and a fully connected layer. 
This architecture is derived from the "w32-10 wide" variant of the Tensorflow model repository. 
The network was trained against an iterative adversary that is allowed to perturb each pixel by at most epsilon=8.0.
"""
None

In [2]:
# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch.nn as nn

import config
import prebuilt_loss_functions as plf
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain


In [3]:
# Step -1: load the classifier, data_loader, and the normalizer 
use_gpu = True
classifier_net = cifar_loader.load_pretrained_cifar_resnet(flavor=32,
                                                           use_gpu=use_gpu)
classifier_net.eval()

val_loader = cifar_loader.load_cifar_data('val', normalize=False, 
                                          use_gpu=use_gpu)

cifar_normer = utils.DifferentiableNormalize(mean=config.CIFAR10_MEANS,
                                             std=config.CIFAR10_STDS)


Files already downloaded and verified


In [4]:
# Step 0: initialize hyperparams for attack and training 
PGD_L_INF = 8.0 / 255.0
PGD_STEP_SIZE = 1.0 / 255.0
PGD_NUM_ITER = 16
PGD_TRAINING_PROP = 1.0

In [5]:
# Step 1: setup loss object 
standard_xentropy = plf.VanillaXentropy(classifier_net, normalizer=cifar_normer)

In [6]:
# Step 2: build attack object and its parameters
pgd_attack_object = aa.LInfPGD(classifier_net, cifar_normer, standard_xentropy)
pgd_attack_params = advtrain.AdversarialAttackParameters(
                            pgd_attack_object, 
                            PGD_TRAINING_PROP,
                            {'attack_kwargs': 
                                {'l_inf_bound': PGD_L_INF,
                                 'step_size': PGD_STEP_SIZE,
                                 'num_iterations': PGD_NUM_ITER,
                                 'random_init': True,
                                 'signed': True,
                                 'verbose': False}})


In [7]:
# Step 3: build training object, training loss, data loader

pgd_cifar_training = advtrain.AdversarialTraining(classifier_net, 
                                                  cifar_normer, 
                                                  'full_pgd_cifar_madry_params',
                                                  'cifar_resnet32')
train_loss = nn.CrossEntropyLoss()
train_loader = cifar_loader.load_cifar_data('train', normalize=False)



Files already downloaded and verified


In [8]:
# Step 4: Do the training 
'''pgd_cifar_training.train(train_loader, 256, train_loss, 
                         attack_parameters=pgd_attack_params,
                         use_gpu=True, verbosity='high')
                         
'''

"pgd_cifar_training.train(train_loader, 256, train_loss, \n                         attack_parameters=pgd_attack_params,\n                         use_gpu=True, verbosity='high')\n                         \n"

In [9]:
"""3/28/18 
Load and verify the adversarially trained model 

"""

import checkpoints
base_model = cifar_resnets.resnet32()
experiment_name = 'full_pgd_cifar_madry_params'
architecture_name = 'cifar_resnet32'
epoch = 100

adv_trained_net = checkpoints.load_state_dict(experiment_name, architecture_name, epoch, base_model)

In [10]:
# Now run the evaluation script on the eval set 

#0) initialize the classifier/normalizer/data loader 
cifar_normer = utils.DifferentiableNormalize(mean=config.CIFAR10_MEANS, std=config.CIFAR10_STDS)
val_loader = cifar_loader.load_cifar_data('val', normalize=False)

#1) Build some attack objects 
L_INF_BOUND = 8.0 / 255.0
# --- FGSM attack
fgsm_xentropy_loss = plf.VanillaXentropy(adv_trained_net,
                                         normalizer=cifar_normer)
fgsm_attack_obj = aa.FGSM(adv_trained_net, cifar_normer,
                          fgsm_xentropy_loss)
fgsm_spec_params = {'attack_kwargs': {'l_inf_bound': L_INF_BOUND, 'verbose': False}}
fgsm_attack_params = advtrain.AdversarialAttackParameters(
                            fgsm_attack_obj, 0.5, fgsm_spec_params)

# --- PGD attack 
pgd_xentropy_loss = plf.VanillaXentropy(adv_trained_net, normalizer=cifar_normer)
pgd_attack_obj = aa.LInfPGD(adv_trained_net, cifar_normer, pgd_xentropy_loss)
pgd_spec_params = {'attack_kwargs': {'l_inf_bound': L_INF_BOUND,
                                     'step_size': 1.0/255.0,
                                     'num_iterations': 16,
                                     'random_init': True,
                                     'signed': True,
                                     'verbose': False}}
pgd_attack_params = advtrain.AdversarialAttackParameters(
                        pgd_attack_obj, 0.5, pgd_spec_params)

# 2) Run the evaluation and print outpus 
eval_obj = advtrain.AdversarialEvaluation(adv_trained_net, cifar_normer)
eval_obj.evaluate(val_loader, {'fgsm': fgsm_attack_params, 'pgd': pgd_attack_params}, 
                  use_gpu=True)

Files already downloaded and verified
Starting minibatch 0...
	 (mb: 0) evaluating pgd...
	 (mb: 0) evaluating fgsm...
Starting minibatch 1...
	 (mb: 1) evaluating pgd...
	 (mb: 1) evaluating fgsm...
Starting minibatch 2...
	 (mb: 2) evaluating pgd...
	 (mb: 2) evaluating fgsm...
Starting minibatch 3...
	 (mb: 3) evaluating pgd...
	 (mb: 3) evaluating fgsm...
Starting minibatch 4...
	 (mb: 4) evaluating pgd...
	 (mb: 4) evaluating fgsm...
Starting minibatch 5...
	 (mb: 5) evaluating pgd...
	 (mb: 5) evaluating fgsm...
Starting minibatch 6...
	 (mb: 6) evaluating pgd...
	 (mb: 6) evaluating fgsm...
Starting minibatch 7...
	 (mb: 7) evaluating pgd...
	 (mb: 7) evaluating fgsm...
Starting minibatch 8...
	 (mb: 8) evaluating pgd...
	 (mb: 8) evaluating fgsm...
Starting minibatch 9...
	 (mb: 9) evaluating pgd...
	 (mb: 9) evaluating fgsm...
Starting minibatch 10...
	 (mb: 10) evaluating pgd...
	 (mb: 10) evaluating fgsm...
Starting minibatch 11...
	 (mb: 11) evaluating pgd...
	 (mb: 11) eva

{'fgsm': 0.2385, 'ground': 0.8226, 'pgd': 0.2014}

In [11]:
20 >= None

True

In [24]:
foo = torch.ones(4, 4, 4)
foo.cpu().numpy()


array([[[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]], dtype=float32)